# 本チャプターの目次
1. データマネジメントとは何者のなのか？
2. メタデータとは？
3. ビッグデータ基盤にメタデータが必要な理由
4. メタデータを保存するメタデータストアとは
5. ビッグデータ基盤におけるメタデータの提供形態
6. 3種類存在するメタデータ
7. 次のセクションへの導入

# データマネジメントとは何者なのか？

データマネジメントとは、

# メタデータとは？


# ビッグデータ基盤にメタデータが必要な理由

# メタデータを保存するメタデータストアとは？

# ビッグデータ基盤におけるメタデータの提供形態

# 3種類存在するメタデータ

# 次セクションへの導入
次のセクションよりメタデータを解説及び、ところどころ算出していきます。
そのための準備を行います。

## データの読み込みとテーブルの作成をします※
1. jinko.csvを読み込んで表示する
2. jinko.csvの内容を保存するテーブルを作成する
3. jinko.csvのデータを2で作成したテーブルにInsertする

※以下の講座で行った最初の準備の簡略版です。
「データサイエンスのための前処理入門PythonとSparkで学ぶビッグデータエンジニアリング(PySpark) 速習講座」
https://www.udemy.com/course/python-spark-pyspark/?referralCode=E67BF8B61F65866794EB


In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init()

In [3]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/19 10:44:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/19 10:44:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/19 10:44:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col

struct = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "SJIS") \
    .csv("/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)
df.show()

+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|          code|    kenmei|gengo|    wareki|   seireki| chu|       sokei|jinko_male|jinko_female|
+--------------+----------+-----+----------+----------+----+------------+----------+------------+
|都道府県コード|都道府県名| 元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|  人口（女）|
|            00|      全国| 大正|         9|      1920|null|    55963053|  28044185|    27918868|
|            01|    北海道| 大正|         9|      1920|null|     2359183|   1244322|     1114861|
|            02|    青森県| 大正|         9|      1920|null|      756454|    381293|      375161|
|            03|    岩手県| 大正|         9|      1920|null|      845540|    421069|      424471|
|            04|    宮城県| 大正|         9|      1920|null|      961768|    485309|      476459|
|            05|    秋田県| 大正|         9|      1920|null|      898537|    453682|      444855|
|            06|    山形県| 大正|         9|      1920|null|      968925|    478328|      49059

In [5]:
spark.sql(""" 
create database if not exists data_management_cruch_course;
""")

21/11/19 10:45:11 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/19 10:45:11 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/19 10:45:12 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
21/11/19 10:45:12 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [6]:
# テーブルの作成をします。
# 今回はCSVに一致した「jinko_table」テーブルを作成します
spark.sql(""" 
CREATE EXTERNAL TABLE IF NOT EXISTS data_management_cruch_course.jinko_table (code String, gengo String,wareki String,seireki String,chu String,sokei String,jinko_male String,jinko_femail String)

PARTITIONED BY (kenmei String)
STORED AS PARQUET
LOCATION '/Users/saitouyuuki/Desktop/src/pyspark_dataprofiling_dataquality/dataset/parquet/';
""")


21/11/19 10:45:18 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
21/11/19 10:45:18 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
21/11/19 10:45:18 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/19 10:45:18 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


DataFrame[]

In [7]:
# 作成したテーブルの一覧を確認することができます
spark.sql("show tables in   data_management_cruch_course").show(truncate=False)

+----------------------------+-----------+-----------+
|namespace                   |tableName  |isTemporary|
+----------------------------+-----------+-----------+
|data_management_cruch_course|jinko_table|false      |
+----------------------------+-----------+-----------+



In [8]:
# テーブルの定義を確認することができます。
spark.sql("show create table data_management_cruch_course.jinko_table").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                                                                                                                                                                                                             |
+-----------------------------------------------------------------------------------------------------------